# Gaussian Graphical Model on a 4‑Cycle
This notebook simulates a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the cyclic graph 1–2–3–4–1.
We verify conditional independence structure and estimate covariance/precision matrices.

## 1. Define the graph and precision matrix

In [8]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  0,  3],
    [ 5, 10,  5,  0],
    [ 0,  5, 10,  5],
    [ 3,  0,  5, 10]
])
Sigma = np.linalg.inv(Theta)
Sigma

array([[ 0.29411765, -0.26470588,  0.23529412, -0.20588235],
       [-0.26470588,  0.38823529, -0.31176471,  0.23529412],
       [ 0.23529412, -0.31176471,  0.38823529, -0.26470588],
       [-0.20588235,  0.23529412, -0.26470588,  0.29411765]])

## 2. Simulate Gaussian sample

In [9]:
N = 10**6
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[ 0.2284922 , -0.23848616,  0.22954781, -0.29600346],
       [-0.69271903,  0.23249835, -0.21619735,  0.36528982],
       [-0.55269577,  0.44440048, -0.34034812,  0.54591197],
       [-0.53004345,  0.71227158, -0.23809721,  0.11444217],
       [-0.19376043, -0.51093053,  1.4735689 , -0.86717736]])

## 4. Estimate covariance and precision

In [10]:
EstimSigma = np.cov(X, rowvar=False)
EstimTheta = np.linalg.inv(EstimSigma)
EstimSigma, EstimTheta

(array([[ 0.29370672, -0.2641558 ,  0.23524747, -0.20581796],
        [-0.2641558 ,  0.387367  , -0.3114421 ,  0.23495567],
        [ 0.23524747, -0.3114421 ,  0.38830817, -0.26470301],
        [-0.20581796,  0.23495567, -0.26470301,  0.29418648]]),
 array([[ 1.00134926e+01,  4.99860912e+00, -8.07443954e-03,
          3.00614684e+00],
        [ 4.99860912e+00,  1.00103948e+01,  5.00329663e+00,
          4.05262853e-03],
        [-8.07443954e-03,  5.00329663e+00,  9.99710181e+00,
          4.99359509e+00],
        [ 3.00614684e+00,  4.05262853e-03,  4.99359509e+00,
          9.99225541e+00]]))

## 5. Maximum-likelihood estimation

In [11]:
from scipy.optimize import minimize

def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

def log_likelihood(theta_vec, S):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    return np.log(det) - np.trace(S @ Theta_mat)



A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]




res = minimize(lambda t: -log_likelihood(t, EstimSigma),
               theta0_vec)

Theta_MLE = construct_symmetric_matrix(res.x)
Theta_MLE

C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


array([[ 1.00135599e+01,  4.99831238e+00, -8.43075986e-03,
         3.00614491e+00],
       [ 4.99831238e+00,  1.00102506e+01,  5.00304869e+00,
         3.62172467e-03],
       [-8.43075986e-03,  5.00304869e+00,  9.99741039e+00,
         4.99392656e+00],
       [ 3.00614491e+00,  3.62172467e-03,  4.99392656e+00,
         9.99321425e+00]])

## 6. Graphical Lasso 


In [21]:
def penalized_log_likelihood(theta_vec, S , lamb):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    sum_Theta = np.absolute(Theta_mat).sum()    
    return np.log(det) - np.trace(S @ Theta_mat )  - lamb * sum_Theta


A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]


lamb = 0.3

res = minimize(lambda t: -penalized_log_likelihood(t, EstimSigma , lamb),
               theta0_vec)

Theta_MLE = construct_symmetric_matrix(res.x)
Theta_MLE  

array([[ 1.68292334e+00, -1.11544328e-10, -9.62022223e-04,
         2.01251609e-03],
       [-1.11544328e-10,  1.46673548e+00,  3.11490064e-02,
        -1.91520942e-03],
       [-9.62022223e-04,  3.11490064e-02,  1.46694374e+00,
         3.82203711e-05],
       [ 2.01251609e-03, -1.91520942e-03,  3.82203711e-05,
         1.67881630e+00]])